In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

[Дообучение](https://github.com/sberbank-ai/ru-gpts/blob/master/examples/Finetune_RuGPTs_with_HF.ipynb) GPT3small на своем наборе данных

# Prepare training dataset

In [ ]:
data=pd.read_csv('data.csv')
data.columns

In [ ]:
#signs=['aries','gemini','taurus','cancer','leo','virgo','libra','scorpio','sagittarius','capricorn','aquarius','pisces']
signs_r=['овен','близнецы','телец','рак','лев','дева','весы','скорпион','стрелец','козерог','водолей','рыбы']
sign_dict={'aries':'овен',
           'gemini':'близнецы',
           'taurus':'телец',
           'cancer':'рак',
           'leo':'лев',
           'virgo':'дева',
           'libra':' весы',
           'scorpio':'скорпион',
           'sagittarius':'стрелец',
           'capricorn':'козерог',
           'aquarius':'водолей',
           'pisces':'рыбы'}

In [ ]:
with open("text.txt", "w", encoding="utf-8") as file:
    for line in data.values:
        sign=sign_dict[line[1]]
        text=line[2]
        
        context='<s>'+sign+'\n' +text+'</s>\n'
        
        file.write(context)
file.close()

In [ ]:
with open("text1.txt", "w", encoding="utf-8") as file:
    for i,line in enumerate(data.values):
        if i==36000:
            break
        sign=sign_dict[line[1]]
        text=line[2]
        
        context='<s>'+sign+'\n' +text+'</s>\n'
        
        file.write(context)
file.close()

with open("text2.txt", "w", encoding="utf-8") as file:
    
    for i,line in enumerate(data.values):
        if i>=36000:
            sign=sign_dict[line[1]]
            text=line[2]

            context='<s>'+sign+'\n' +text+'</s>\n'

            file.write(context)
file.close()


In [ ]:
text=data.values[:,2]
vals=np.zeros((text.shape[0]))
for i,line in enumerate(text):
    vals[i]=len(line.split())
print(vals.max())
print(vals.mean())
print(vals.std())

In [ ]:
plt.figure(figsize=(10,10))
plt.hist(vals,bins=50)

# Text generation

In [ ]:
#signs=['aries','gemini','taurus','cancer','leo','virgo','libra','scorpio','sagittarius','capricorn','aquarius','pisces']
signs_r=['овен','близнецы','телец','рак','лев','дева','весы','скорпион','стрелец','козерог','водолей','рыбы']
sign_dict={'aries':'овен',
           'gemini':'близнецы',
           'taurus':'телец',
           'cancer':'рак',
           'leo':'лев',
           'virgo':'дева',
           'libra':' весы',
           'scorpio':'скорпион',
           'sagittarius':'стрелец',
           'capricorn':'козерог',
           'aquarius':'водолей',
           'pisces':'рыбы'}

In [ ]:
model_path='models/astro_gpt3large'
tok = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)
model.cuda()

In [ ]:
for sig in signs_r:

    text = "<s> "+sig
    inpt = tok.encode(text, return_tensors="pt")
    out = model.generate(inpt.cuda(), 
                         max_length=100
                         ,
                         repetition_penalty=5.0,
                         
                         do_sample=True, top_k=5,
                         top_p=0.95,
                         temperature=1)
    predict=tok.decode(out[0])
    print(sig.capitalize()+'\n'+str(predict).capitalize()+'\n')